In [2]:
import pyccl as ccl
import scipy.integrate as sp
import scipy.special as spec
import numpy as np
import matplotlib.pyplot as plt
import csv
from scipy.optimize import curve_fit
from scipy.interpolate import InterpolatedUnivariateSpline as spline
from sympy.physics.wigner import wigner_3j 
import time
from mpmath import hyp2f1
import pyfftlog as fftl


In [13]:
def bhi(z):
    b_HI0 = 0.677105 # amplitude of bias function
    
    return (b_HI0 / 0.677105) * (6.6655e-01 + 1.7765e-01*z + 5.0223e-02*z**2.)

In [15]:
h = 0.67
cosmo = ccl.Cosmology(Omega_c=0.315, Omega_b=0.049, h=h, n_s=0.96, sigma8=0.83)
#z_i = 1
#z_i = 0.3915
z_i = 0.0415

# h = 0.705 # comparison with GiggleZ sims xi2d
# cosmo = ccl.Cosmology(Omega_c=0.273, Omega_b=0.0456, h=0.705, n_s=0.96, sigma8=0.812)
# z_i = 0.6

def power(k,z):
    return bhi(z)**2*ccl.linear_matter_power(cosmo, k, 1/(1+z))


##########
#z_i = 1
#z_i = 0.6
#z_i = 0.3915 # mid MK upper
##########

COFFEDfacsq = 0.368150

#################################
k_arr = np.logspace(-6, 2, 10001)
#################################

k_nobao = np.array([k for k in k_arr if (k>0.45 or k<0.017)])
psmo = spline(np.log(k_nobao), np.log([power(k,z_i) for k in k_nobao]), k=3)
#psmo = spline(np.log(k_nobao), np.log([COFFEDfacsq*power(k,0) for k in k_nobao]), k=3) # for coffe comparison at z=1 

psmo_actual = spline(k_arr, [np.exp(psmo(k)) for k in np.log(k_arr)],k=1)
ar1 = np.exp([psmo(k) for k in np.log(k_arr)])
ar2 = [power(k,z_i) for k in k_arr]
#ar2 = [COFFEDfacsq*power(k,0) for k in k_arr] # for coffe comparison at z=1

prelim = np.array([ar2[i]/ar1[i] for i in range(len(ar1))])


fbao = spline(k_arr, prelim - 1)

def power2(k,alpha,A):
    return (1+A*fbao(alpha*k))*psmo_actual(k)

In [321]:
def chi2(data, mv, cinv):
    return np.matmul( (data - mv).T ,  np.matmul( cinv , (data - mv) ) )

In [57]:
def fit_corr_point(r,alpha,A,R):
    if R != 0:
        yarray= 1/(r*R) * np.sin(k_arr*r) * (1+A*fbao(alpha*k_arr))*psmo_actual(k_arr) * spec.dawsn(k_arr*R) 
    else:
        yarray= k_arr/r * np.sin(k_arr*r) * (1+A*fbao(alpha*k_arr))*psmo_actual(k_arr) 
    return sp.simps(yarray, x=k_arr) /(2*np.pi**2)

In [118]:
def fit1_corr_point(r,alpha,A,R):
    " BASE "
    
    """meshgrid vectorised version for scipy curve fit 
    
     IMPORTANT - if using the output in 1D e.g. to generate a mean vector,
                 the output must use output.squeeze() to prevent complaints """
    
    r2d, k2d = np.meshgrid(r, k_arr)
    
    if R != 0:
        yarray= 1/(r2d*R) * np.sin(k2d*r2d) * power2(k2d,alpha,A) * spec.dawsn(k2d*R) /(2*np.pi**2)

    else:

        yarray= k2d/r2d * np.sin(k2d*r2d) * power2(k2d,alpha,A) /(2*np.pi**2)
        

    return np.array([sp.simps(y, x=k_arr)  for y in yarray.T])

In [119]:
def fit1_quad_point(r,alpha,A,R):
    " BASE "
    
    """meshgrid vectorised version for scipy curve fit 
    
     IMPORTANT - if using the output in 1D e.g. to generate a mean vector,
                 the output must use output.squeeze() to prevent complaints """
    
    r2d, k2d = np.meshgrid(r, k_arr)
    
    if R != 0:
        yarray= -1*k2d/R  * spec.spherical_jn(2, k2d*r2d) * power2(k2d,alpha,A) * spec.dawsn(k2d*R) /(2*np.pi**2)

    else:

        yarray= -1*k2d**2 * spec.spherical_jn(2, k2d*r2d) * power2(k2d,alpha,A) /(2*np.pi**2)
        

    return np.array([sp.simps(y, x=k_arr)  for y in yarray.T])

In [120]:
def fit1_hdp_point(r,alpha,A,R):
    " BASE "
    
    """meshgrid vectorised version for scipy curve fit 
    
     IMPORTANT - if using the output in 1D e.g. to generate a mean vector,
                 the output must use output.squeeze() to prevent complaints """
    
    r2d, k2d = np.meshgrid(r, k_arr)
    
    if R != 0:
        yarray= -k2d/R  * spec.spherical_jn(4, k2d*r2d) * power2(k2d,alpha,A) * spec.dawsn(k2d*R) /(2*np.pi**2)

    else:

        yarray= k2d**2 * spec.spherical_jn(4, k2d*r2d) * power2(k2d,alpha,A) /(2*np.pi**2)
        

    return np.array([sp.simps(y, x=k_arr)  for y in yarray.T])

In [50]:
def beam_21cm(D):
    "takes dish diameter in metres and returns theta_FWHM in RADIANS"
    
    return 0.211 / D

def rco(z):
    "comoving distance to z"
    return ccl.comoving_radial_distance(cosmo, 1/(1+z))

def beamwidth(z, theta, ef=1):
    "Calculates the comoving factor R_beam as a function of z. requires theta in RADIANS"
    return rco(z) * theta * (1+z) * ef/(8*np.log(2))**0.5

In [4]:
asadx, asady = [900, 1300], [1.21, 1.16]
asad = spline(asadx, asady, k=1, ext=0)

def asadfactor(z):
    "extra holographics factor on the beam size - given as a function of frequency" 
    " in the Asad et al. MeerKAT Holographics paper"
    
    vem = 1420
    
    vobs = vem / (1+z)
        
    return asad(vobs)

In [327]:
def Rbeam(rp,R):
    "function of r_perp - remember that!"
    "real space gaussian beam transverse size R"
    return np.exp(-0.5*(rp/R)**2)

def agbeamHT(k,R):
    "analytic gaussian beam Hankel Transform - B(k) "
    return np.exp(-0.5*(k*R)**2)

def dawsonbeam(k,R):
    "note: this is the *squared* HT beam factor, after a monopole!"
    return spec.dawsn(k*R) / (k*R)

In [385]:
"MeerKAT L beam, mid band 0.3915"
MKL_HTf = np.load('/home/hera/notebooks/Kennedy/MKL_HT.npy')
MKL_HT = spline(MKL_HTf[:,0],MKL_HTf[:,1],k=1)

    

In [73]:
def HT_stretch_beam(beamfile,z, Nk, Ddish=13.5):
    "Hankel Transform of z-stretched real space beam from actual (in function) to target FWHM of MeerKAT"
    "beamfile should be numpy array!"
    
    R = beamwidth(z, beam_21cm(Ddish), asadfactor(z))
    print('R ',R)
    target = R * (8*np.log(2))**0.5 /2 # converting to HWHM, the 0.5 point
    print('target ',target)
    "root of the beam function spline dropped by 0.5 = HWHM"
    actual = spline(beamfile[:,0], beamfile[:,1] - 0.5,k=3).roots()
    print('actual ',actual)
    
    realspacebeam = spline(  beamfile[:,0] * target/actual , beamfile[:,1], k=1,ext=1)
    
    k_ = np.logspace(-4,1,Nk)

    lim = beamfile[-2,0] * target/actual # Limit for r-range of Hankel Transform integral, to avoid ringing.
                                         # (I'm taking the 2nd to last point in case of rounding) 
       
    norm_factor = HT(realspacebeam, 0, lim)
    
    return spline(k_ , HT(realspacebeam, k_, lim) / norm_factor , k=1 )
    

In [72]:
def HT(func, k_, lim):
    "Hankel Transform of a function of r_perp, generating a function of K_perp"
    r_ = np.linspace(0,lim[0],30000) 
    
    r2d,k2d = np.meshgrid(r_,k_)
    
    y_arr = func(r2d) * r2d * spec.j0(r2d*k2d) 
    return np.array([sp.trapz(y, x=r_) for y in y_arr])

In [9]:
def T_b(z):
    """
    HI brightness temperature, in K.
    """
    return (5.5919e-02 + 2.3242e-01*z - 2.4136e-02*z**2.) / 1e3 # K

In [14]:
def rsd(m,z):
    "RSD power as a function of mu (cos theta)"

    f = ccl.growth_rate(cosmo, 1/(1+z)) 
    
    
    return (1 + f/bhi(z)*m**2)**2 

In [388]:
def rsd_fit(m,z,bhi,f):
    "RSD power as a function of mu (cos theta)"
    return (bhi + f*m**2)**2 

In [371]:
def rsd_c(m,z):
    "RSD power as a function of mu (cos theta)"
    
    b_HI0 = 0.677105 # amplitude of bias function
    
    #bhi = (b_HI0 / 0.677105) * (6.6655e-01 + 1.7765e-01*z + 5.0223e-02*z**2.)
    bhi = 1.
    
    ######################################################
    
    f = 0.5545
    #f = ccl.growth_rate(cosmo, 1/(1+z)) 
    
    
    return (bhi + f*m**2)**2 

In [33]:
def rsd_fog(k,m,z):
    "RSD power as a function of k and mu (cos theta) - includes FOG"
    bhi = 1
    sigmav = 4e2
    H0 = 68
    f = ccl.growth_rate(cosmo, 1/(1+z))
    beta = f**0.55 / bhi
    
    return 0.5*(bhi**2 * (1+beta*m**2)**2) / (1+ (k*m*sigmav / H0)**2)

In [337]:
def pole(ell,z,k_,func1=1,p1=1,func2=1,p2=1,weight_fn=None, p3=1, kfg=0.1, nmu=5000):
    "monopole of a function of K_perp and/or K, mu to given powers p1 & p2"
    "takes z as an argument, but at present only RSD depends on this, the beams must be HT's at redshift z"
    
    "--I COULD USE FFTLOG FOR THE HT HERE! MAKE IT MUCH FASTER!--"
    
    "func1 is a function of K_perp (i.e. a beam function)"
    "func2 is a function of k,mu (i.e. RSD)"
    
    mu_ = np.linspace(-1,1,nmu)
    
    k2d, m2d = np.meshgrid(k_,mu_)
    

    if func1 == 1:
        F1 = 1
    else:
        F1 =  func1( k2d * (1-m2d**2)**0.5 )** p1  
    if func2 == 1:
        F2 = 1
    else:
        F2 =  func2(m2d,z) ** p2
    
    W = 1.
    if weight_fn is not None:
        W = weight_fn(k2d,m2d,kfg) **p3
        
    yarr = spec.eval_legendre(ell,m2d) * F1 * F2 * W
    
    return (2*ell + 1)/2 * np.array([sp.trapz(y, x=mu_) for y in yarr.T])

In [320]:
def polepw(ell,z,k_,powerf,a_perp,a_par,A,func1=1,func2=1,weight_fn=None, kfg=0.1, nmu=5000,cl='cl',pt=0):
    "powerf is a parameterised power spectrum {k,mu,alpha_perp,alpha_par,Amplitude}"
    "func1 is a function of K_perp (i.e. a beam function)"
    "func2 is a function of k,mu (i.e. RSD)"
    "weight fn is the FG cut function"
    
    if pt==1:print('POLE %d'%ell)
    
    if cl=='cl':
        p1,p2,p3,p4 = 1,2,1,1
    if cl=='cltilde':
        p1,p2,p3,p4 = 2,4,2,2
    
    mu_ = np.linspace(-1,1,nmu)
    mu_half = mu_[:int(nmu/2)] # speedup for symmetric angular arguments (functions of mu^2) 
    
    k2d, m2d = np.meshgrid(k_,mu_)
    k2d_h, m2d_h = np.meshgrid(k_,mu_half)
    
    t0 = time.time()
    if pt==1:print('evaluating power spectrum model.. ', end=' ')
    P = np.empty((nmu,len(k_))) 
    p_h = powerf(k2d_h,m2d_h,a_perp,a_par,A) **p1  # speedup for symmetric angular arguments (functions of mu^2) 
    P[:int(nmu/2),:] = p_h
    P[int(nmu/2):,:] = np.flip(p_h,0)
    
    t1 = time.time()
    if pt==1: print('%.4f'%(t1-t0))
    if pt==1: print('evaluating other angular functions.. ', end=' ')
        
    if func1 == 1:
        BM = 1
    else:
        BM = np.empty((nmu,len(k_)))  
        b_h = func1( k2d_h * (1-m2d_h**2)**0.5 )** p2   
        BM[:int(nmu/2),:] = b_h
        BM[int(nmu/2):,:] = np.flip(b_h,0)
    
    if func2 == 1:
        RSD = 1
    else:
        RSD =  func2(m2d,z) ** p3      
    
    W = 1.
    if weight_fn is not None:
        W = weight_fn(k2d,m2d,kfg) **p4   

    t2 = time.time()
    if pt==1: print('%.4f'%(t2-t1))
    if pt==1: print('building integrand.. ', end=' ')

        
    yarr = spec.eval_legendre(ell,m2d) * P * BM * RSD * W
    
    t3 = time.time()
    if pt==1: print('%.4f'%(t3-t2))
    if pt==1: print('evaluating trapezium rule.. ', end=' ')
    
    pole = (2*ell + 1)/2 * np.array([sp.trapz(y, x=mu_) for y in yarr.T])
    
    t4 = time.time()
    if pt==1: print('%.4f'%(t4-t3))
    
    return pole

In [317]:
def powerf(k,mu,a_perp,a_par,A):
    " assumes meshgrid input with k[0] == 1D k array "
    return (1+A*fbao( ( (a_perp*k)**2*(1-mu**2) + (a_par*k*mu)**2 )**0.5 ))*psmo_actual(k[0])[np.newaxis,:]

In [305]:
def corrmpfit(r_0, r_2, z, alpha, A, R, C, D, E, F, G, H, FGcut=None, RSD=1, Nkpole=8000, nmu=1000, pt=1, fit='joint'):
    "monopole and quadrupole FFTLog fitting function"
    
    t1 = time.time()
    if pt==1: print('calculating poles...', end=' ')
    "passing params to pole() to generate cl's L=0,2 "
    if R!=0: beamsw=lambda x: agbeamHT(x, R) 
    else: beamsw = 1
    if FGcut!=None: weightsw=weight_fn 
    else: weightsw=None
    if RSD==1: rsdsw=rsd 
    else: rsdsw=1
            
    kcl_ = np.logspace(-6,2,Nkpole)
    
    p0 = pole(0, z, kcl_, beamsw, 2, rsdsw, 1, weightsw, 1, FGcut, nmu, pt)
    p2 = pole(2, z, kcl_, beamsw, 2, rsdsw, 1, weightsw, 1, FGcut, nmu, pt)
    
    t2 = time.time()
    if pt==1:print('%.4f'%(t2-t1))
    if pt==1:print('splining cls...', end=' ')
    
    cl0 = spline(kcl_, p0, k=1)
    cl2 = spline(kcl_, p2, k=1)
    
    
    t3 = time.time()
    if pt==1:print('%.4f'%(t3-t2))
    if pt==1:print('FFTLog...', end=' ')
    
    "FFTLog"

    prefac = (np.pi/2)**0.5 /(2*np.pi**2)
    
    logkmin, logkmax = -6,2
    n = 2048
    nc = (n + 1)/2.0
    kr = 0.0008
    logkc = (logkmin + logkmax)/2
    dlogk = (logkmax - logkmin)/n
    dlnk = dlogk*np.log(10.0)
    k = 10**(logkc + (np.arange(1, n+1) - nc)*dlogk)
    
    ak0 = k * cl0(k) * power2(k,alpha,A)
    ak2 = k * cl2(k) * power2(k,alpha,A)
    
    kr0, xsave0 = fftl.fhti(n, 1/2, dlnk, 1/2, kr, 1) # kropt = 1
    logrc = np.log10(kr0) - logkc
    rk = 10**(logkc - logrc)
    corr0 = prefac*fftl.fhtq(ak0.copy(), xsave0, 1)
    r0 = 10**(logrc + (np.arange(1, n+1) - nc)*dlogk)
    
    kr2, xsave2 = fftl.fhti(n, 5/2, dlnk, 1/2, kr, 1) # kropt = 1
    logrc = np.log10(kr2) - logkc
    rk2 = 10**(logkc - logrc)
    corr2 = -1* prefac* fftl.fhtq(ak2.copy(), xsave2, 1)
    r2 = 10**(logrc + (np.arange(1, n+1) - nc)*dlogk)
    
    t4 = time.time()
    if pt==1:print('%.4f'%(t4-t3)) 
    if pt==1:print('complete %.4f'%(time.time()-t1))
    
    mono = r_0**-2*spline(r0, corr0, k=1)(r_0) + C + D/r_0 + E/r_0**2
    quad = r_2**-2*spline(r2, corr2, k=1)(r_2) + F + G/r_2 + H/r_2**2
    
    if fit=='joint':
        return np.concatenate((mono,quad))
    elif fit=='mono':
        return mono
    elif fit=='quad':
        return quad
    else:
        print("fit parameter should be 'joint', 'mono', or 'quad'")

In [315]:
def weight_fn(k, mu, kfg):
    "weight function - gaussian edge"
    kpar = k * mu # assumes meshgrid input
    return 1. - np.exp(-0.5 * (kpar / kfg)**2.)

In [366]:
def corrmpfitpw(r_b, ic, z, a_perp, a_par, A, R, C, D, E, F, G, H, I, powerf=powerf, FGcut=None, RSD=1, Nkpole=1000, nmu=500, pt=0, fit='joint'):
    "monopole and quadrupole FFTLog fitting function"
    
    t1 = time.time()
    if pt==1: print('calculating pole0...', end=' ')
    "passing params to pole() to generate cl's L=0,2 "
    
    if R=='MKL': beamsw=MKL_HT
    elif R>0: beamsw=lambda x: agbeamHT(x, R)
    else: beamsw = 1
    if FGcut!=None: weightsw=weight_fn 
    else: weightsw=None
    if RSD==1: rsdsw=rsd 
    else: rsdsw=1
            
    kcl_ = np.logspace(-6,2,Nkpole)
    
    p0 = polepw(0, z, kcl_, powerf, a_perp, a_par, A, beamsw, rsdsw, weightsw, FGcut, nmu, cl='cl',pt=pt)
    t2 = time.time()
    if pt==1:print('%.4f'%(t2-t1))
    if pt==1: print('calculating pole2...', end=' ')
    
    p2 = polepw(2, z, kcl_, powerf, a_perp, a_par, A, beamsw, rsdsw, weightsw, FGcut, nmu, cl='cl',pt=pt)
    
    t3 = time.time()
    if pt==1:print('%.4f'%(t3-t2))
    if pt==1:print('splining cls...', end=' ')
    
    cl0 = spline(kcl_, p0, k=1)
    cl2 = spline(kcl_, p2, k=1)
    
    
    t4 = time.time()
    if pt==1:print('%.4f'%(t4-t3))
    if pt==1:print('FFTLog...', end=' ')
    
    "FFTLog"

    prefac = (np.pi/2)**0.5 /(2*np.pi**2)
    
    logkmin, logkmax = -6,2
    n = 2048
    nc = (n + 1)/2.0
    kr = 0.0008
    logkc = (logkmin + logkmax)/2
    dlogk = (logkmax - logkmin)/n
    dlnk = dlogk*np.log(10.0)
    k = 10**(logkc + (np.arange(1, n+1) - nc)*dlogk)
    
    ak0 = k * cl0(k) 
    ak2 = k * cl2(k) 
    
    kr0, xsave0 = fftl.fhti(n, 1/2, dlnk, 1/2, kr, 1) # kropt = 1
    logrc = np.log10(kr0) - logkc
    rk = 10**(logkc - logrc)
    corr0 = prefac*fftl.fhtq(ak0.copy(), xsave0, 1)
    r0 = 10**(logrc + (np.arange(1, n+1) - nc)*dlogk)
    
    kr2, xsave2 = fftl.fhti(n, 5/2, dlnk, 1/2, kr, 1) # kropt = 1
    logrc = np.log10(kr2) - logkc
    rk2 = 10**(logkc - logrc)
    corr2 = -1* prefac* fftl.fhtq(ak2.copy(), xsave2, 1)
    r2 = 10**(logrc + (np.arange(1, n+1) - nc)*dlogk)
   
    r_0 = r_b[:ic]
    r_2 = r_b[ic:]
    
    #mono = r_0**-2*spline(r0, corr0, k=1)(r_0) + C + D/r_0 + E/r_0**2
    mono = r_0**-2*spline(r0, corr0, k=1)(r_0) + C + D/r_0 + E/r_0**2 + I*r_0
    quad = r_2**-2*spline(r2, corr2, k=1)(r_2) + F + G*r_2 + H*r_2**2
    #quad = r_2**-2*spline(r2, corr2, k=1)(r_2) + F + G*r_2 + H/r_2

    t5 = time.time()
    if pt==1:print('%.4f'%(t5-t4)) 
    if pt==1:print('complete %.4f'%(time.time()-t1))
    
    if fit=='joint':
        return np.concatenate((mono,quad))
    elif fit=='mono':
        return mono
    elif fit=='quad':
        return quad
    else:
        print("fit parameter should be 'joint', 'mono', or 'quad'")

In [333]:
def corrfn2d_old(cosmo, z, r, theta, coeffs, Nk=1000):
    
    ells = np.arange(len(coeffs))
    a = 1. / (1. + z)
    
    k = np.logspace(-4., 2., Nk)
    pk = ccl.nonlin_matter_power(cosmo, k, a)
    integ = k**2. * pk
    
    # Integral over k
    kr = r[np.newaxis,:] * k[:,np.newaxis]
    I_ell = np.array([sp.simps(((coeffs[ell] * integ)[:,np.newaxis] * spec.spherical_jn(ell, kr)).T, k) for ell in ells])
    #print(I_ell.shape)
    
    # Angle-dependent prefactor
    kappa = np.cos(theta)
    xi = 0
    for i, ell in enumerate(ells):
        xi += (1.j)**ell * I_ell[i,:,np.newaxis] * spec.legendre(ell)(kappa)[np.newaxis,:]
    xi /= (2.*np.pi**2.)
    return xi

In [343]:
def corrfn2d(r, z, theta, cl, Nk=1000, nmu=500):
    "FFTLog generation of multipoles of xi; then summation w/ Legendre coeffs"
    
             # just sort out the arguments and this function is ready to go
    
    if R=='MKUHF': beamsw=MKUHF_HT
    elif R>0: beamsw=lambda x: agbeamHT(x, R)
    else: beamsw = 1
    if FGcut!=None: weightsw=weight_fn 
    else: weightsw=None
    if RSD==1: rsdsw=rsd 
    else: rsdsw=1
            
    kcl_ = np.logspace(-6,2,Nk)
    
    mu_ = np.linspace(-1,1,nmu)
    
    cls,xis = [],[]
    
    for ell in range(0,maxell,2):
        cls.append(spline(kcl_, polepw(ell, z, kcl_, powerf, 1, 1, 1, beamsw, rsdsw, weightsw, FGcut, nmu, cl='cl'), k=1))
       
    prefac = (np.pi/2)**0.5 /(2*np.pi**2)
    
    for ell in range(0,maxell,2):
        fftl_ = prefac*(-1)**(ell/2)*FFTLog(r_x2d,ell,cls[int(ell/2)])
        xis.append(fftl_[:,np.newaxis] * spec.eval_legendre(ell, mu_)[np.newaxis,:])
                                     
    xi = np.zeros((len(r),nmu))
    for x in xis:
        xi+=x
    
    
    return xi

In [286]:
def multipole_cl(ell,r,cl):
    " BASE "
    
    "currently not a function of z - will need to be for fitting"
    
    """meshgrid vectorised version for scipy curve fit 
    
     IMPORTANT - if using the output in 1D e.g. to generate a mean vector,
                 the output must use output.squeeze() to prevent complaints """
    
    
    r2d, k2d = np.meshgrid(r, k_arr)
    

    yarray=  k2d**2 * spec.spherical_jn(ell, k2d*r2d) * cl(k2d) * power2(k2d,1,1) 


    return ((1.j)**ell /(2*np.pi**2)).real * np.array([sp.simps(y, x=k_arr)  for y in yarray.T])


In [335]:
def covmp(r, ell1, ell2, cl, cltilde, nbar, Vol, Nk=50000):
    
    """compute the covariance (0,0), (0,2), (2,2) of the correlation function subject to any modulation effects 
    presented in the given CLs. the argument 'r' should be an array.
    
    The CLs should be given as a list containing the monopole, quadrupole and hexadecapole contribution as numpy 
    arrays to be splined over (i.e. functions of k, computed by the pole function). The reason for only these 
    contributions is that these are the only nonzero Wigner_3j symbols in the summation term, when considering up
    to the (2,2) covariance"""
    
    pixelsize=r[1]-r[0] # assumes linear spacing of R values
    
    s = len(r) # side length

    k_ = np.logspace(-6,2,Nk)   

    K_int = np.empty((s,s))

    "splining CL's"
    t0 = time.time()
    print('splining cls..', end=' ')

    cl0 = spline(cl[0][:,0],cl[0][:,1],k=1)
    cl2 = spline(cl[1][:,0],cl[1][:,1],k=1)
    cl4 = spline(cl[2][:,0],cl[2][:,1],k=1)
    clt0 = spline(cltilde[0][:,0],cltilde[0][:,1],k=1)
    clt2 = spline(cltilde[1][:,0],cltilde[1][:,1],k=1)
    clt4 = spline(cltilde[2][:,0],cltilde[2][:,1],k=1)

    "because I only need (0,0),(0,2),(2,2) , this section is sadly hardcoded!"
    t1 = time.time()
    print('%.4fs'%(t1-t0))
    print('summing P(k) c_L(k) W terms..', end=' ')

    if ell1+ell2==0:
        P_CL1 = 2/nbar * power2(k_,1,1) * cl0(k_)
        P_CL2 = power2(k_,1,1)**2 *clt0(k_)
    if ell1+ell2==2:
        P_CL1 = 2/nbar * power2(k_,1,1) * cl2(k_) / 5
        P_CL2 = power2(k_,1,1)**2 * clt2(k_) / 5
    if ell1+ell2==4:
        P_CL1 = 2/nbar * power2(k_,1,1) * (cl0(k_)/5 +  (2/35)*( cl2(k_) + cl4(k_) ))
        P_CL2 = power2(k_,1,1)**2 * ( clt0(k_)/5 +  (2/35)*( clt2(k_) + clt4(k_) ) )

    
#     P_CL1 = 2/nbar * power2(k_,1,1)
#     P_CL2 = power2(k_,1,1)**2          # for the purposes of comparison - RSD will just be added as constants
#                                     # will still need to check that the above c_l's do the same thing..!
    
    "building the covariance"
    t2 = time.time()
    print('%.4fs'%(t2-t1))
    print('building cov side %d..'%s)
    K_int = np.empty((s,s))

    for i in range(s):
        print('%d'%(i+1), end=' ')
        for j in range(s):
            r1,r2 = r[i],r[j]
            y_ = k_**2 * spec.spherical_jn(ell1,k_*r1) * spec.spherical_jn(ell2,k_*r2) * (P_CL1 + P_CL2)
            K_int[i,j] = (2*ell1 + 1)*(2*ell2+1)*(sp.simps(y_, k_))

    "adding diagonal terms"
    t3 = time.time()
    print('')
    print('built %.4fs'%(t3-t2))
    print('adding diagonal..', end=' ')

    diag = 0
    if ell1==ell2:
        diag = np.zeros((s,s))
        for i in range(s):
            diag[i,i] = (2*ell1+1)*np.pi / (2*pixelsize*nbar**2*r[i]**2)


    SUM = diag + K_int

    COV = (1.j)**(ell1 - ell2) / (Vol * np.pi**2) * SUM
    e = time.time()
    print('%.3fs'%(e-t3))
    print('complete  %.4fs'%(e-t0))

    return COV.real

In [151]:
def I_ell(ell, nu, t):

    """

    I_ell(nu, t) function from Eq. 2.20 of arXiv:1705.05022.

    """

    fac = 2.**(nu-1.) * np.pi**2. * spec.gamma(ell+0.5*nu)  / (spec.gamma(0.5*(3. - nu)) * spec.gamma(ell + 1.5))

    y = fac * t**ell * hyp2f1(0.5*(nu-1.), ell+0.5*nu, ell+1.5, t**2.)

    return complex(y)

In [152]:
def fftlog(fn, kmin=1e-3, kmax=0.3, b=0, nsamp=128):
    """
    Calculate log-space FFT coefficients according to Eq. 2.14 of 
    arXiv:1705.05022.
    """
    # Get log-space k values

    logk = np.linspace(np.log(kmin), np.log(kmax), nsamp)
    deltalnk = np.log(kmax/kmin)
    k = np.exp(logk)

    # Evaluate non-Bessel factors in integrand
    f = fn(k) * np.exp((3. + b)*logk)

    # Perform FFT to get c_n coefficients
    coeff = np.fft.fft(f) # FIXME: May not have the right scaling
    n = np.fft.fftfreq(nsamp) * nsamp

    return coeff / deltalnk, n

In [167]:
def fftlog_direct(fn, kmin=1e-3, kmax=0.3, b=0, nsamp=64, intsamp=400):
    """
    Calculate log-space FFT coefficients according to Eq. 2.14 of 
    arXiv:1705.05022.
    """
    # Get log-space k values
    logk = np.linspace(np.log(kmin), np.log(kmax), intsamp)
    deltalnk = np.log(kmax/kmin)
    k = np.exp(logk)

    # Evaluate non-Bessel factors in integrand
    f = fn(k) * np.exp((3. + b)*logk)

    # Perform direct FT to get c_n coefficients
    nvals = np.fft.fftfreq(nsamp)*nsamp
    cn = []
    for n in nvals:
        integ = fn(k) * np.exp((3.+b)*logk)   * np.exp(-2.*np.pi*1.j * n * logk / deltalnk)
        y = sp.simps(integ, logk)
        cn.append(y)
        
    return np.array(cn) / deltalnk, nvals

In [180]:
def integrate_2bessel(fn, ell, d, dp, kmin=1e-3, kmax=0.3, b=0, nsamp=128):
    """
    Integrate function with double spherical Bessel in integrand, using Eqs. 
    2.18 -- 2.20 of arXiv:arXiv:1705.05022.
    """

    # Define function in integrand and get FFT log transform

    cn, nn = fftlog(fn, kmin=kmin, kmax=kmax, b=b, nsamp=nsamp)

    # Make sure chi >= r
    chi, r = (d, dp) if d > dp else (dp, d)

    # Calculate integral using hypergeometric result
    nfac = 2.*np.pi*1.j / np.log(kmax/kmin) # nu_n = nfac*n - b
    integ = 0
    
    for i, n in enumerate(nn):
        #print(I_ell(ell=ell, nu=nfac*n - b, t=r/chi), nfac*n - b, r/chi) # DEBUG
        integ += cn[i] * chi**(-(nfac*n - b)) * I_ell(ell=ell, nu=nfac*n - b, t=r/chi)

        
    integ /= (4.*np.pi)

    return complex(integ)

In [323]:
def chisq(x, dof, scale):
    return scale*(2**dof * spec.gamma(dof/2) ) **-1 * x**(dof/2 - 1) * np.exp(-x/2)

def chisq_test(x, dof, loc, scale):
    return scipychi2.pdf(x, dof,loc,scale)

def fch2(dat,bins=20):
    "fits a chi2 to an output set of chi2's - returns the fit degrees of freedom"
    
    "PROBLEM - Cannot fit distributions with d.o.f >~ 100, for these values it will return "
    "a d.o.f guess of exactly the p0 value. In these cases the function will now assign the mode"
    "of the histogram"
    
    xs,ys = np.histogram(dat, bins=bins)[1], np.histogram(dat, bins=bins)[0]
    xs = [(xs[i]+xs[i+1])/2 for i in range(bins)]
    try:
        opt, cv = curve_fit(chisq, xs, ys, p0=[32,20], maxfev=100000)
    except RuntimeError:
        return xs[np.where(ys==max(ys))[0][0]]
    if opt[0] == 32.000 or opt[0]==0.000:
    
        return xs[np.where(ys==max(ys))[0][0]]
    return opt[0]


def fch2test(dat,bins=20):
    "TEST FUNCTION - fits a chi2 to an output set of chi2's - returns the fit degrees of freedom"
    xs,ys = np.histogram(dat, bins=bins)[1], np.histogram(dat, bins=bins)[0]
    xs = [(xs[i]+xs[i+1])/2 for i in range(bins)]
    opt, cv = curve_fit(chisq, xs, ys, p0=[32,1e20], maxfev=100000)
    return opt[0], opt[1]

In [362]:
def numerical_res(wp, location, bins=20):
    
    name = location+'/R%.2f_f%d_RA%.2f_sim-%s_chi-%s_%dsims.npy'%(wp[0][3],wp[3],wp[7],wp[4],wp[5],wp[6])
    try:
        res = np.load(name)
    except FileNotFoundError:
        return 0, 0, 0, 0, 0

        
    alphapps = res[:,0] 
    alphapas = res[:,1] 
    chidfs = res[:,2] 

    alphapps_mean = np.mean(alphapps)
    alphapps_sigma = np.std(alphapps)
    
    alphapas_mean = np.mean(alphapas)
    alphapas_sigma = np.std(alphapas)
    
    chi2df = fch2(chidfs, bins)

    return alphapps_mean, alphapps_sigma, alphapas_mean, alphapas_sigma, chi2df

In [344]:
def FFTLog(r_0, ell, cl):
    logkmin, logkmax = -6,2
    n = 2048
    nc = (n + 1)/2.0
    kr = 0.0008
    logkc = (logkmin + logkmax)/2
    dlogk = (logkmax - logkmin)/n
    dlnk = dlogk*np.log(10.0)
    k = 10**(logkc + (np.arange(1, n+1) - nc)*dlogk)
    
    ak = k * cl(k) 
     
    kr, xsave = fftl.fhti(n, ell+1/2, dlnk, 1/2, kr, 1) # kropt = 1
    logrc = np.log10(kr) - logkc
    rk = 10**(logkc - logrc)
    corr = fftl.fhtq(ak.copy(), xsave, 1)
    r = 10**(logrc + (np.arange(1, n+1) - nc)*dlogk)
            
    res = r_0**-2*spline(r, corr, k=1)(r_0) 
    return res

In [389]:
def polepw_rsd(ell,z,k_,powerf,a_perp,a_par,A,bhi, f,func1=1,func2=1,weight_fn=None, kfg=0.1, nmu=5000,cl='cl',pt=0):
    "powerf is a parameterised power spectrum {k,mu,alpha_perp,alpha_par,Amplitude}"
    "func1 is a function of K_perp (i.e. a beam function)"
    "func2 is a function of k,mu (i.e. RSD)"
    "weight fn is the FG cut function"
    
    if pt==1:print('POLE %d'%ell)
    
    if cl=='cl':
        p1,p2,p3,p4 = 1,2,1,1
    if cl=='cltilde':
        p1,p2,p3,p4 = 2,4,2,2
    
    mu_ = np.linspace(-1,1,nmu)
    mu_half = mu_[:int(nmu/2)] # speedup for symmetric angular arguments (functions of mu^2) 
    
    k2d, m2d = np.meshgrid(k_,mu_)
    k2d_h, m2d_h = np.meshgrid(k_,mu_half)
    
    t0 = time.time()
    if pt==1:print('evaluating power spectrum model.. ', end=' ')
    P = np.empty((nmu,len(k_))) 
    p_h = powerf(k2d_h,m2d_h,a_perp,a_par,A) **p1  # speedup for symmetric angular arguments (functions of mu^2) 
    P[:int(nmu/2),:] = p_h
    P[int(nmu/2):,:] = np.flip(p_h,0)
    
    t1 = time.time()
    if pt==1: print('%.4f'%(t1-t0))
    if pt==1: print('evaluating other angular functions.. ', end=' ')
        
    if func1 == 1:
        BM = 1
    else:
        BM = np.empty((nmu,len(k_)))  
        b_h = func1( k2d_h * (1-m2d_h**2)**0.5 )** p2   
        BM[:int(nmu/2),:] = b_h
        BM[int(nmu/2):,:] = np.flip(b_h,0)
    
    if func2 == 1:
        RSD = 1
    else:
        RSD =  func2(m2d,z,bhi,f) ** p3      
    
    W = 1.
    if weight_fn is not None:
        W = weight_fn(k2d,m2d,kfg) **p4   

    t2 = time.time()
    if pt==1: print('%.4f'%(t2-t1))
    if pt==1: print('building integrand.. ', end=' ')

        
    yarr = spec.eval_legendre(ell,m2d) * P * BM * RSD * W
    
    t3 = time.time()
    if pt==1: print('%.4f'%(t3-t2))
    if pt==1: print('evaluating trapezium rule.. ', end=' ')
    
    pole = (2*ell + 1)/2 * np.array([sp.trapz(y, x=mu_) for y in yarr.T])
    
    t4 = time.time()
    if pt==1: print('%.4f'%(t4-t3))
    
    return pole

In [390]:
def corrmpfitpw_rsd(r_b, ic, z, a_perp, a_par, A, R, C, D, E, F, G, H, I,bhi,f, powerf=powerf, FGcut=None, RSD=1, Nkpole=1000, nmu=500, pt=0, fit='joint'):
    "monopole and quadrupole FFTLog fitting function"
    
    t1 = time.time()
    if pt==1: print('calculating pole0...', end=' ')
    "passing params to pole() to generate cl's L=0,2 "
    
    if R=='MKL': beamsw=MKL_HT
    elif R>0: beamsw=lambda x: agbeamHT(x, R)
    else: beamsw = 1
    if FGcut!=None: weightsw=weight_fn 
    else: weightsw=None
    if RSD==1: rsdsw=rsd_fit
    else: rsdsw=1
            
    kcl_ = np.logspace(-6,2,Nkpole)
    
    p0 = polepw_rsd(0, z, kcl_, powerf, a_perp, a_par, A, bhi,f, beamsw, rsdsw, weightsw, FGcut, nmu, cl='cl',pt=pt)
    t2 = time.time()
    if pt==1:print('%.4f'%(t2-t1))
    if pt==1: print('calculating pole2...', end=' ')
    
    p2 = polepw_rsd(2, z, kcl_, powerf, a_perp, a_par, A,bhi,f, beamsw, rsdsw, weightsw, FGcut, nmu, cl='cl',pt=pt)
    
    t3 = time.time()
    if pt==1:print('%.4f'%(t3-t2))
    if pt==1:print('splining cls...', end=' ')
    
    cl0 = spline(kcl_, p0, k=1)
    cl2 = spline(kcl_, p2, k=1)
    
    
    t4 = time.time()
    if pt==1:print('%.4f'%(t4-t3))
    if pt==1:print('FFTLog...', end=' ')
    
    "FFTLog"

    prefac = (np.pi/2)**0.5 /(2*np.pi**2)
    
    logkmin, logkmax = -6,2
    n = 2048
    nc = (n + 1)/2.0
    kr = 0.0008
    logkc = (logkmin + logkmax)/2
    dlogk = (logkmax - logkmin)/n
    dlnk = dlogk*np.log(10.0)
    k = 10**(logkc + (np.arange(1, n+1) - nc)*dlogk)
    
    ak0 = k * cl0(k) 
    ak2 = k * cl2(k) 
    
    kr0, xsave0 = fftl.fhti(n, 1/2, dlnk, 1/2, kr, 1) # kropt = 1
    logrc = np.log10(kr0) - logkc
    rk = 10**(logkc - logrc)
    corr0 = prefac*fftl.fhtq(ak0.copy(), xsave0, 1)
    r0 = 10**(logrc + (np.arange(1, n+1) - nc)*dlogk)
    
    kr2, xsave2 = fftl.fhti(n, 5/2, dlnk, 1/2, kr, 1) # kropt = 1
    logrc = np.log10(kr2) - logkc
    rk2 = 10**(logkc - logrc)
    corr2 = -1* prefac* fftl.fhtq(ak2.copy(), xsave2, 1)
    r2 = 10**(logrc + (np.arange(1, n+1) - nc)*dlogk)
   
    r_0 = r_b[:ic]
    r_2 = r_b[ic:]
    
    #mono = r_0**-2*spline(r0, corr0, k=1)(r_0) + C + D/r_0 + E/r_0**2
    mono = r_0**-2*spline(r0, corr0, k=1)(r_0) + C + D/r_0 + E/r_0**2 + I*r_0
    quad = r_2**-2*spline(r2, corr2, k=1)(r_2) + F + G*r_2 + H*r_2**2
    #quad = r_2**-2*spline(r2, corr2, k=1)(r_2) + F + G*r_2 + H/r_2

    t5 = time.time()
    if pt==1:print('%.4f'%(t5-t4)) 
    if pt==1:print('complete %.4f'%(time.time()-t1))
    
    if fit=='joint':
        return np.concatenate((mono,quad))
    elif fit=='mono':
        return mono
    elif fit=='quad':
        return quad
    else:
        print("fit parameter should be 'joint', 'mono', or 'quad'")

In [8]:
ccl.h_over_h0(cosmo, 1/(1+0.3915)) 

1.2716090855519968